In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import random

In [2]:
all_df = pd.read_csv(r"C:\Users\hvvel\Downloads/original+bt_data.csv",engine='python')

In [3]:
all_df.shape

(93692, 3)

In [6]:
all_df.head

<bound method NDFrame.head of        Unnamed: 0                                          text_data  \
83747       83747           I've never seen a pussy she didn't like.   
17121       17121    Black bitches be like boy i told you not to ...   
36966       36966  Young bitches talk about their new man ALLLL D...   
54407       54407                             UserAbd is Muslim Scum   
13394       13394     ironholds This is the average copyright tex...   
...           ...                                                ...   
46814       46814                what i was talking to usersandstein   
15879       15879                      Trust no player fear no bitch   
23891       23891  If I call you a little ch then you hate me pro...   
66676       66676    September  UTC Disagree amicably Did you rea...   
71075       71075  The comparison of Shotokan and Wado is foolish...   

       profane_class  
83747              1  
17121              1  
36966              1  
54407        

In [5]:
all_df = all_df.sample(frac=1)

In [7]:
# remove stopwords
# pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hvvel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
all_df["text_data"] = all_df.text_data.map(remove_stopwords)

In [9]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(all_df.text_data)

In [10]:
len(counter)

121005

In [11]:
counter.most_common(5)

[('article', 15856),
 ('bitch', 15341),
 ('page', 13855),
 ('like', 13767),
 ('wikipedia', 10984)]

In [12]:
num_unique_words = len(counter)

In [13]:

# Split dataset into training and test set
random.seed(100)
train_size = int(all_df.shape[0] * 0.7)

train_df = all_df[:train_size]
test_df = all_df[train_size:]

# split text and labels
train_sentences = train_df.text_data.to_numpy()
train_labels = train_df.profane_class.to_numpy()
test_sentences = test_df.text_data.to_numpy()
test_labels = test_df.profane_class.to_numpy()

In [14]:
train_sentences.shape, test_sentences.shape

((65584,), (28108,))

In [15]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [16]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [17]:
print(train_sentences[0:5])
print(train_sequences[0:5])

["i've never seen pussy like."
 'black bitches like boy told get hair wet tcocsgoopgky'
 'young bitches talk new man allll dayas anyone really intrested shit lol'
 'userabd muslim scum'
 'ironholds average copyright text organisation offers informations statistics strictly allows use informations contain source accused stupidity cant read phrases like copyright unicef copyright unicef unicef reserves copyrights material web pages citation told dont understand copyright issues makes say']
[[1261, 77, 281, 32, 4], [109, 21, 4, 656, 403, 14, 1068, 1985, 41441], [903, 21, 8, 46, 142, 22071, 41442, 162, 44, 22072, 17, 38], [41443, 1700, 2660], [14575, 1828, 223, 198, 2907, 5335, 7055, 2738, 3551, 2198, 26, 7055, 1701, 64, 1454, 4013, 80, 86, 3435, 4, 223, 13306, 223, 13306, 13306, 12232, 3662, 308, 520, 54, 675, 403, 7, 180, 223, 396, 243, 39]]


In [18]:
len_sequences = []
for one_seq in train_sentences:
    word_list = one_seq.split() 
    #print(len(word_list))
    len_sequences.append(len(word_list))
#print(len_sequences)
pd.Series(len_sequences).describe()

count    65584.000000
mean        21.579852
std         42.769526
min          0.000000
25%          6.000000
50%         10.000000
75%         19.000000
max       1250.000000
dtype: float64

In [19]:

# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 50

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, test_padded.shape

((65584, 50), (28108, 50))

In [20]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 32)            3872160   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 3,897,057
Trainable params: 3,897,057
Non-trainable params: 0
_________________________________________________________________


In [21]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [22]:
model.fit(train_padded, train_labels, epochs=20, validation_split=0.1, verbose=2)

Epoch 1/20
1845/1845 - 78s - loss: 0.1771 - accuracy: 0.9326 - val_loss: 0.1162 - val_accuracy: 0.9581
Epoch 2/20
1845/1845 - 76s - loss: 0.0857 - accuracy: 0.9727 - val_loss: 0.1278 - val_accuracy: 0.9532
Epoch 3/20


ResourceExhaustedError:  OOM when allocating tensor with shape[121005,32] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node Adam/Adam/update/AssignVariableOp (defined at <ipython-input-22-7e07ba3da378>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_3254]

Function call stack:
train_function


In [ ]:
predictions = model.predict(test_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

In [ ]:
print(test_sentences[10:20])

print(test_labels[10:20])
print(predictions[10:20])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, predictions)

In [ ]:
from sklearn.metrics import precision_score
precision_score(test_labels, predictions)

In [ ]:
from sklearn.metrics import recall_score
recall_score(test_labels, predictions)